In [11]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd
import numpy as np

import matplotlib.pyplot  as plt

In [12]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
         database='SMartDB', user='postgres',
      password='P@ssw0rd', host='localhost', 
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
    
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

In [17]:
"""
TO_CHAR(inventory.customer_warranty_start  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as customer_warranty_start,
TO_CHAR(inventory.customer_warranty_end AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as customer_warranty_end,

 'customer_warranty_start','customer_warranty_end'

"""

start_date_query='2020-01-01'
end_date_query='2023-12-31'
#severity.severity_name as  severity_label,
sql_all="""
select
incident.id as id, incident.incident_no as incident_no,

severity.id as  severity_id,
severity.severity_name as  severity_name,


service_level.sla_name as sla,

product_type.productype_name as product_type,brand.brand_name as brand,model.model_name as model,



xtype.incident_type_name as incident_type,
status.incident_status_name as status,
service.service_type_name service_type,
CASE WHEN failure_type IS NULL THEN  0 ELSE 1 END AS is_failure_type,

(select count(*) from  app_incident_detail  as detail where  detail.incident_master_id=incident.id ) as count_detail


,TO_CHAR(incident.incident_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as open_datetime
,TO_CHAR(incident.incident_close_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as close_datetime

,TO_CHAR(incident.incident_problem_start  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as response_datetime
,TO_CHAR(incident.incident_problem_end  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as resolved_datetime

,company.company_name as company



from app_incident as incident
inner join app_incident_type as  xtype on incident.incident_type_id = xtype.id
inner join  app_incident_status as status on incident.incident_status_id = status.id
inner join  app_incident_severity as severity on  incident.incident_severity_id = severity.id
inner join  app_service_type as service on incident.service_type_id= service.id

inner join app_inventory as inventory on incident.inventory_id = inventory.id

inner join app_brand as brand on inventory.brand_id = brand.id
inner join app_model as model on inventory.model_id = model.id
inner join app_product_type as product_type on inventory.product_type_id = product_type.id
inner join app_sla as service_level on inventory.customer_sla_id = service_level.id

inner join app_project as project on inventory.project_id = project.id
inner join app_company as company on project.company_id = company.id

where incident.incident_status_id =4
and incident.incident_datetime>=%(start_date_param)s
and  incident.incident_datetime<=%(end_date_param)s


order by incident.incident_datetime desc


"""

In [14]:
print("Create all issues dataframe")

dict_params={"start_date_param":start_date_query,"end_date_param":end_date_query}

df_all=list_data(sql_all,dict_params,get_postgres_conn())

# convert object to datetime
dateTimeCols=['open_datetime','response_datetime','resolved_datetime','close_datetime']
for col in dateTimeCols:
 df_all[col]=pd.to_datetime(df_all[col], format='%Y-%m-%d %H:%M',errors= 'coerce')

df_all.dropna(inplace=True)

print(df_all.info())
#df_all[df_all['close_datetime'].isna()]

df_all.isnull().sum()

Create all issues dataframe
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2195 entries, 0 to 2194
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 2195 non-null   int64         
 1   incident_no        2195 non-null   object        
 2   severity_id        2195 non-null   int64         
 3   severity_name      2195 non-null   object        
 4   sla                2195 non-null   object        
 5   product_type       2195 non-null   object        
 6   brand              2195 non-null   object        
 7   model              2195 non-null   object        
 8   incident_type      2195 non-null   object        
 9   status             2195 non-null   object        
 10  service_type       2195 non-null   object        
 11  is_failure_type    2195 non-null   int64         
 12  count_detail       2195 non-null   int64         
 13  open_datetime      2195 non-null   

id                   0
incident_no          0
severity_id          0
severity_name        0
sla                  0
product_type         0
brand                0
model                0
incident_type        0
status               0
service_type         0
is_failure_type      0
count_detail         0
open_datetime        0
close_datetime       0
response_datetime    0
resolved_datetime    0
company              0
dtype: int64

In [15]:
df_all.tail()

,id,incident_no,severity_id,severity_name,sla,product_type,brand,model,incident_type,status,service_type,is_failure_type,count_detail,open_datetime,close_datetime,response_datetime,resolved_datetime,company
2190,574,SR-ES-22-0574,3,Minor,24x7 6Hrs Resolution Time,Storage,YIP,dummy,General Incident,Closed,Request,0,1,2020-05-22 16:54:00,2020-05-25 12:09:00,2020-05-25 12:09:00,2020-05-25 12:09:00,BBL
2191,573,SR-ES-22-0573,2,Major,24x7 4Hrs Response Time,Storage,NetApp,AFF A800,General Incident,Closed,Incident,0,3,2020-05-14 13:48:00,2020-05-29 19:07:00,2020-05-14 16:26:00,2020-05-21 17:27:00,BBL
2192,572,SR-ES-22-0572,2,Major,24x7 4Hrs Response Time,Storage,NetApp,AFF A800,General Incident,Closed,Incident,0,2,2020-04-23 14:58:00,2020-04-24 11:08:00,2020-04-23 15:31:00,2020-04-23 20:03:00,BBL
2193,571,SR-ES-22-0571,2,Major,24x7 4Hrs Response Time,Storage,NetApp,AFF A800,General Incident,Closed,Incident,0,1,2020-04-16 16:26:00,2020-04-17 16:13:00,2020-04-17 16:13:00,2020-04-17 16:13:00,BBL
2194,570,SR-ES-22-0570,2,Major,24x7 4Hrs Response Time,Storage,NetApp,AFF A800,General Incident,Closed,Incident,0,2,2020-03-15 13:36:00,2020-03-16 20:45:00,2020-03-15 15:03:00,2020-03-16 19:31:00,BBL


In [16]:
# start_end_list=[ ['open_datetime','close_datetime'],\
#                 ['response_datetime','resolved_datetime']\
#                ]
# listDiffHourCols=[]
# for item  in  start_end_list:
#    diff_str=f"{item[0]}_to_{item[1]}" 

#    diff_str=diff_str.replace('_datetime','')  
#    df_all[diff_str]=df_all[item[1]]-df_all[item[0]]
    
#    diff_hour=f'{diff_str}_hour'
#    listDiffHourCols.append(diff_hour)
#    df_all[diff_hour] = df_all[diff_str].apply(lambda x:  x.total_seconds() / (60*60) if x is not np.nan else np.nan  )

# df_all[listDiffHourCols].describe()

In [34]:
# plt.hist(df_all['open_to_close_hour'])
# plt.show()